In [1]:
'''
Image and Data Generator
'''

from imutils import paths
import numpy as np
import pandas as pd
import os
import cv2
from keras.applications import VGG16, imagenet_utils
from keras.preprocessing.image import img_to_array, load_img

path = "flickr8k/flickr_data/Flickr_Data/Images/"
imagePaths = os.listdir(path)
mappings = "flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr8k.lemma.token.txt"

#Loading all the caotions and data--------------------


images = []
captions = []
with open(mappings, 'r') as file:
    for line in file:
    
        data = line.split('\t')
        images.append(data[0])
        captions.append(data[1])
        
#------------------------------------------------------
        
#-----Loading the available data------------------------       
mat = []
precap = []
for i,img in zip(captions[:4000:], images[:4000:]):
    if img[:-2:] in imagePaths:
        image = load_img(path+img[:-2:], target_size=(224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)
        mat.append(image)
        precap.append(i)  
        
#---------------------------------------------------------    
model = VGG16(weights='imagenet', include_top=False)

bs = 16
Total_feature = []
for i in range(0,len(mat),bs):
    batchImages = mat[i:i+bs]
    batchImages = np.vstack(batchImages)
    feature = model.predict(batchImages, batch_size=bs)
    Total_feature.append(feature)
print(len(Total_feature))


Using TensorFlow backend.








250


In [2]:
# Total_feature = Total_feature[:312:]

In [2]:
data = []
for batch in Total_feature:
#     print(batch.shape)
    batch_data = []
    for i in range(16):
        temp = np.asarray(batch[i,::,::,::])
        data.append(temp.reshape(7*7*512))
        
data = np.asarray(data)
data.shape

(4000, 25088)

In [21]:
precap = precap[:4000:]

In [22]:
print('Total_Image','Total_Captions','Total_Images_using','Total_Captions_to_be_used', sep='\t')
print(len(images), len(captions), len(precap), len(data), sep='\t')

Total_Image	Total_Captions	Total_Images_using	Total_Captions_to_be_used
40460	40460	4000	4000


In [23]:
import nltk
import string
stopwords = nltk.corpus.stopwords.words('english') #FUNCTION TO CLEAR STOPWORDS
def text_process(mess):
    '''
    1.Clear the punctuations.
    2.Clear the stop words
    3.Return list of clean words

    '''
    nopunc = [c for c in str(mess) if c not in string.punctuation]

    nopunc = ''.join(nopunc)
    clean_mess = [c for c in nopunc.split() if c.casefold() not in stopwords and c not in stopwords ]
    clean_mess = ' '.join(clean_mess)
    clean_mess = "<start> " + clean_mess + " <end>"
    return clean_mess
precap_cleaned = [text_process(i) for i in precap]

In [24]:
print(precap_cleaned)

['<start> man street racer armor examine tire another racer motorbike <end>', '<start> Two racer drive white bike road <end>', '<start> Two motorist ride along vehicle oddly design color <end>', '<start> Two person small race car drive green hill <end>', '<start> Two person race uniform street car <end>', '<start> firefighter extinguish fire hood car <end>', '<start> fireman spray water hood small white car jack <end>', '<start> fireman spray inside open hood small white car jack <end>', '<start> fireman use firehose car engine carjack <end>', '<start> Firefighter use water extinguish car fire <end>', '<start> boy sand surf hill <end>', '<start> man attempt surf hill make sand sunny day <end>', '<start> man slide huge sand dune sunny day <end>', '<start> man surf hill sand <end>', '<start> young man short tshirt snowboard bright blue sky <end>', '<start> kid play blue tub full water outside <end>', '<start> hot day three small kid sit big container fill water <end>', '<start> Little ki

In [25]:
precap_cleaned = [i.lower() for i in precap_cleaned]
chars = list(set(precap_cleaned))
text_data = []
chars = []
for i in precap_cleaned:
    sentence = i.split()
    for word in sentence:
        text_data.append(word)
        if word not in chars:
            chars.append(word)
            
chars = sorted(chars)
char_to_ix = {ch:i+1 for i,ch in enumerate(chars)}
ix_to_chars = { i+1:ch for i, ch in enumerate(chars)}


In [46]:
char_to_ix['<start>']

20

In [26]:
vocab_size = len(ix_to_chars)
maxLen = len(max(precap_cleaned, key=len).split())
print(maxLen, vocab_size)

17 1910


In [27]:

def read_glove(path):

    word_to_vec_map = {}
    words = set()
    with open(path, 'r') as f:
        for line in f:
            currline = line.strip().split()
            currWord = currline[0]
            words.add(currWord)
            currFeature = currline[1:]
            word_to_vec_map[currWord]= np.array(currFeature, dtype=np.float64)

        word_to_index = {}
        index_to_word = {}
        i=1
        for word in sorted(words):
            word_to_index[word] = i
            index_to_word[i] = word
            i+=1


        return word_to_index, index_to_word, word_to_vec_map

In [28]:
word_to_index, index_to_word, word_to_vec_map = read_glove('/home/aniket/bruce/DeepLearning/glove.6B/glove.6B.200d.txt')

In [29]:
embedding_dim = 200

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in char_to_ix.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]  = embedding_vector

In [30]:
embedding_matrix

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.38273  , -0.057823 , -0.71589  , ...,  0.37161  , -0.61747  ,
         0.062199 ],
       [-0.88293  ,  0.50863  , -1.1317   , ...,  0.0038354,  0.36563  ,
        -0.06873  ],
       ...,
       [ 0.28974  ,  0.091194 ,  0.79137  , ...,  1.0182   , -0.61807  ,
         0.27235  ],
       [ 0.16932  , -0.55324  ,  0.11445  , ..., -0.64154  ,  0.015441 ,
        -0.58962  ],
       [-0.30765  ,  0.603    , -0.33111  , ...,  0.23741  , -0.44581  ,
        -0.55649  ]])

In [31]:
from keras.models import Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Add, Activation
from keras import Input
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [32]:
inputs1 = Input(shape=(25088,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape = (maxLen,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 17)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 25088)        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 17, 200)      382000      input_5[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 25088)        0           input_4[0][0]                    
____________________________________________________________________________________________

In [33]:
model.layers[2]

In [34]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [36]:
precap_cleaned

['<start> man street racer armor examine tire another racer motorbike <end>',
 '<start> two racer drive white bike road <end>',
 '<start> two motorist ride along vehicle oddly design color <end>',
 '<start> two person small race car drive green hill <end>',
 '<start> two person race uniform street car <end>',
 '<start> firefighter extinguish fire hood car <end>',
 '<start> fireman spray water hood small white car jack <end>',
 '<start> fireman spray inside open hood small white car jack <end>',
 '<start> fireman use firehose car engine carjack <end>',
 '<start> firefighter use water extinguish car fire <end>',
 '<start> boy sand surf hill <end>',
 '<start> man attempt surf hill make sand sunny day <end>',
 '<start> man slide huge sand dune sunny day <end>',
 '<start> man surf hill sand <end>',
 '<start> young man short tshirt snowboard bright blue sky <end>',
 '<start> kid play blue tub full water outside <end>',
 '<start> hot day three small kid sit big container fill water <end>',
 '

In [37]:
from numpy import array as array
def data_generator(captions, images_vec, word_to_index, maxlength, np):
    
    X1, X2, y = list(), list(), list()
    n=0
    while 1:
        for i,caption in enumerate(captions):
            n+=1
            img = images_vec[i]
            seq = [char_to_ix[word] for word in caption.split(' ') if word in char_to_ix]
            
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                
                in_seq = pad_sequences([in_seq], maxlen=maxlength)[0]
                
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                
                X1.append(img)
                X2.append(in_seq)
                y.append(out_seq)
                
                
            if n == np:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0
                
                   
                

In [39]:
epochs = 5
num = 3
steps = len(precap_cleaned)//num

In [40]:
for i in range(epochs):
    generator = data_generator(precap_cleaned, data, char_to_ix, maxLen, num)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
model.save('model_'+str(i)+'.h5')

Epoch 1/1
1333/1333 [==============================] - 151s 113ms/step - loss: 13.6598
Epoch 1/1
1333/1333 [==============================] - 149s 112ms/step - loss: 13.6586
Epoch 1/1
1333/1333 [==============================] - 149s 112ms/step - loss: 13.6586
Epoch 1/1
1333/1333 [==============================] - 153s 115ms/step - loss: 13.6586
Epoch 1/1
1333/1333 [==============================] - 154s 116ms/step - loss: 13.6586


In [ ]:
from keras.models import load_model
loaded_model = load_model('model_4.h5')
loaded_model.optimizer.lr = 0.0001
epochs = 10
num = 6
steps = len(precap_cleaned)//num

for i in range(epochs):
    generator = data_generator(precap_cleaned, data, char_to_ix, maxLen, num)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

Epoch 1/1
204/666 [========>.....................] - ETA: 1:06 - loss: 13.7455

In [24]:
model.optimizer.lr = 0.0001
epochs = 10
num = 6
steps = len(precap)//num

for i in range(epochs):
    generator = data_generator(precap, data, char_to_ix, maxLen, num)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

Epoch 1/1
183/832 [=====>........................] - ETA: 1:25 - loss: 16.1181

KeyboardInterrupt: 

In [44]:
def greedySearch(photo):
    in_text = '<start>'
    for i in range(maxLen):
        sequence = [char_to_ix[w] for w in in_text.split() if w in char_to_ix]
        print(sequence)
        sequence = pad_sequences([sequence], maxlen=maxLen)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        print(yhat)
        word = ix_to_chars[yhat]
#         print(word)
        in_text+= ' ' + word
        print(in_text)
        if word == '<end>':
            break
          
    
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final
            

In [45]:
dog = path+images[6110][:-2:]

dog = load_img(dog, target_size=(224, 224))
dog = img_to_array(dog)
dog = np.expand_dims(dog, axis=0)
dog = imagenet_utils.preprocess_input(dog)

imagemodel = VGG16(weights='imagenet', include_top=False)
dog_feature = imagemodel.predict([dog])

dog_feature = [i.reshape(7*7*512) for i in dog_feature]
dog_feature = np.asarray(dog_feature)
dog_feature.shape
print("Greedy:", greedySearch(dog_feature))

[20]
19
<start> <end>
Greedy: 


In [ ]:
loaded_model.save('loaded_model.h5')

In [ ]:
import cv2
dog = path+images[8000][:-2:]
dog = cv2.imread(dog)
cv2.imshow(dog)


In [ ]:

X1=[]
X2=[]
y=[]

for i,caption in enumerate(precap_cleaned):
#             n+=1
#             img = images_vec[i]
            seq = [char_to_ix[word] for word in caption.split(' ') if word in char_to_ix]
#          
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                print(in_seq, out_seq)
                in_seq = pad_sequences([in_seq], maxlen=33)[0]
                print(len(in_seq))
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                X1.append(img)
                X2.append(in_seq)
                y.append(out_seq)

In [ ]:
print(y[0].shape)